In [ ]:
import requests
from urllib.parse import urlencode
import json
import csv
import pandas as pd
from geopy.distance import geodesic 
import folium

In [ ]:
api_key = ''

In [ ]:
#Cleaning Data 

In [ ]:
data = pd.read_csv (r'path.csv')
data.head(5)

In [ ]:
population = pd.read_csv (r'path.csv')
population.head(5)

In [ ]:
data.rename({"Store name":"StoreName"}, axis="columns", inplace=True)
data.rename({"Postal Code":"PostalCode"}, axis="columns", inplace=True)
data.rename({"Store Category1":"StoreCategory1"}, axis="columns", inplace=True)
data.rename({"Store Category2":"StoreCategory2"}, axis="columns", inplace=True)
data.rename({"Store Category3":"StoreCategory3"}, axis="columns", inplace=True)
data['Category1'] = data['StoreCategory1'].str.split('category/').str[1]
data['Category2'] = data['StoreCategory2'].str.split('category/').str[1]
data['Category3'] = data['StoreCategory3'].str.split('category/').str[1]
data = data.drop(['StoreCategory1', 'StoreCategory2', 'StoreCategory3'], axis=1)
data.Category1 = data.Category1.str.replace('+', ', ')
data.Category2 = data.Category2.str.replace('+', ', ')
data.Category2 = data.Category3.str.replace('+', ', ')
data.Address = data.Address.str.replace('\n', '')
data.PostalCode = data.PostalCode.str.replace(' ', '')
data.PostalCode = data.PostalCode.str.replace('-', '')
data.head(5)

# Mapping Competitiors and Potential Location

In [ ]:
def mapping(potential_location_postal_code):
    endpoint = f'https://maps.googleapis.com/maps/api/geocode/json'
    params = {'address': 'potential_location_postal_code', 'key': api_key}
    url_params = urlencode(params)
    url = f'{endpoint}?{url_params}'
    r = requests.get(url)
    latlng = {}
    try:
        latlng = r.json()['results'][0]['geometry']['location']
    except:
        pass
    potential_location_latlng = (latlng.get('lat'), latlng.get('lng'))
    
    data['Distance'] = data.apply(lambda x:geodesic((x["Latitude"], x["Longitude"]), potential_location_latlng), axis=1)
    
    Radius_10km = data.loc[(data['Distance'] < 10.00)]
    
    New_Data = Radius_10km[Radius_10km["Category1"].str.contains("Furniture|Kitchen")]
    
    Location_Data = New_Data.sort_values(by=['Distance'], ascending=True)
    
    New_map = folium.Map(location=[potential_location_latlng], zoom_start=12)
    
    # instantiate a feature group for the competitors in the dataframe
    competitors = folium.map.FeatureGroup()

    # loop through the competitiors and add each to the competitors feature group
    for lat, lng, in zip(New_Data.Latitude, New_Data.Longitude):
        competitors.add_child(
            folium.CircleMarker(
                [lat3, lng3],
                radius=5, # define how big you want the circle markers to be
                color='yellow',
                fill=True,
                fill_color='blue',
                fill_opacity=0.6
            )
         )

    # add pop-up text to each marker on the map
    latitudes = list(New_Data.Latitude)
    longitudes = list(New_Data.Longitude)
    labels = list(New_Data.StoreName)

    for lat, lng, label in zip(latitudes, longitudes, labels):
        folium.Marker([lat, lng], popup=label).add_to(New_map)    
    
    # add competitors to map
    New_map.add_child(competitors)

    # add potential location to map
    folium.Marker(
       location=[potential_location_latlng],
       popup='Potential Location',
       icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(New_map)

    Competitiors_in_10km = New_Data['StoreName'].count()
    print("Number of competitors in 10 km radius:", Competitiors_in_10km3)
    print("Population:", population[population.postcode == potential_location_postal_code].to_string(index=False))
    New_map
